## Lecture 15 Decision Trees

### Decision Trees

| id | Age | Smoker | FamilyHx | Disease (y) |
|---:|----:|:------:|:--------:|:-----------:|
| 1  | 30  |   no   |    no    |      0      |
| 2  | 37  |  yes   |    no    |      0      |
| 3  | 41  |   no   |   yes    |      0      |
| 4  | 46  |  yes   |    no    |      1      |
| 5  | 50  |   no   |   yes    |      1      |
| 6  | 52  |  yes   |   yes    |      1      |
| 7  | 58  |   no   |    no    |      0      |
| 8  | 63  |  yes   |    no    |      1      |

Let $X$ be training data set.